### Linear Programming

Linear programming is the attempt to maximize (or minimize) expressions of the form:
$$
   \max_{\vec{x}} \vec{c}^T \vec{x}
$$
under the constraints
$$
    A \vec{x} \leq \vec{b}
$$
with $\vec{x} \in \mathbb{R}^n$

The solution space of $\vec{x}$ may also be constrainted individually. Often, they are subjected to
$$
    x_i \geq 0
$$

### The simplex method

#### In Text
Notice that each constraint partitions the valid solution space into two in a straight line/plane. The set of constraints would create a polygon that contains the possible solutions.

For the maximized expression is also linear (ie, there is a line/plane where the expression is constant), there is a constant direction of maximum ascent (the direction vector for the line, normal for the plane). This implies that the maximizing solution will be on a vertex, where two or more constraints intersect (proof for reader).

The simplex method pivots along each vertex in the solution boundary, moving along the constraint edges using "loose" variables. Assuming the positive constraints on the parameters, the algorithm is
- Phrase the constraint inequalities as equalities by adding loose variables to the left, re-arange the constraints with only the loose variables on the left. The tight variables is the solution space parameter $\vec{x}$
- Choose the highest, non-negative coefficient in the optimizing expression, and loosen its respective parameter variable
- For each constraint having the then loosened parameter, select the one with the highest non-positive constant to the loosened variable coefficient ratio that is negative. Tighten the loose parameter for that constraint
- Re-arrange the constraints to only have the loose parameter on the left. Update the optimizing expression to have parameters of only tight variables
- Terminate when all the coefficients in the optimizing expression is negative. The vertex coordinate is computed by setting all tight variables to zero


#### Mathematically

Our goal is then to turn the general linear program into a canonical form.

Define the linear program tableau as
$$
\begin{bmatrix}
     1 & -c^T & 0\\
     0 & A & b
\end{bmatrix}
$$
for the linear program
$$
\begin{align*}
     \max_{x}(&c^T x)\\
     Ax &= b\\
     x &\geq 0
\end{align*}
$$

To convert them, we supply each inequality with a slack variable
$$
     ax \leq 2 \implies ax + s = 2, s \geq 0
$$
this creates the equalities and also guarantees the canonical form of the tableau.

We can only apply the simplex method on a canonical tableu, namely one where there forms a semi-identity matrix within $A$.
In such cases, the column variables (indices of $x$) they represent are called basic variables, while the other columns are
called nonbasic variables. By setting the nonbasis variables zero, we regain the current vertex.

We can read the linear program tableau
$$
\begin{bmatrix}
     k & -c^T & Z \\
     0 & A & b
\end{bmatrix}
$$
as maximizing the function $Z = \frac{1}{k} (c^T x)$, with the constraints $Ax = b$. Name the first maximizing row the operational row.

To apply the simplex method on the linear program tableau, we follow the steps
- Select an entering variable from the nonbasic variables, it will become a basic variable. This corrosponds to selecting a tableau column
- Select an non-operational row based on the leaving variable algorithm: with the row with the column element $a_{rc}$ positive and that is the minimum under $b_r / a_{rc}$. This ensures that the basic variable will be positive.
- Normalize the row against $a_{rc}$ by dividing against it. Zero all other elements in rows at this column using row operations.
- The result is an exchange of a nonbasic element with a basic element.
- Carry on until the operational row cannot be improved.

The enter variable selection algorithm is to choose the one with the largest derivative (ie coefficient under the scheme). The termination condition is an objective function that cannot be improved (all negative coef for maximization).

How this works is borderline magic.

### Binary Variables


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from dataclasses import dataclass

@dataclass
class Constraint:
    """
    Represents a constraint in a linear program
    """
    left: dict[str, float]
    right: float
    gt: bool = False

    def __str__(self) -> str:
        out = []
        for key, value in self.left.items():
            if value == 0:
                continue

            if value == 1:
                out.append(key)
            else:
                out.append(f"{value} {key}")

        left = ' + '.join(out)

        symbol = '>=' if self.gt else '<='
        return f'{left} {symbol} {self.right}'


@dataclass
class Maximized:
    """
    Represents an objective function in a linear program
    """
    expression: dict[str, float]
    constant: float

    def __str__(self) -> str:
        out = []
        for key, value in self.expression.items():
            if value == 0:
                continue

            if value == 1:
                out.append(key)
            else:
                out.append(f"{value} {key}")

        left = ' + '.join(out)

        if self.constant == 0:
            return left
        
        return f'{left} + {self.constant}'

    def vars(self) -> list[str]:
        return list(self.expression.keys())

@dataclass
class LPProblem:
    """
    Represents a linear program with inequalities and an objective function
    """

    constraints: list[Constraint]
    maximized: Maximized

    def __str__(self) -> str:
        lines = []
        lines.append('LPProblem')
        lines.append('')

        lines.append('Constraints:')
        for c in self.constraints:
            lines.append(str(c))
        
        lines.append('')
        lines.append('Maximizing:')
        lines.append(str(self.maximized))

        return '\n'.join(lines)
    
    def add(self, c: Constraint):
        self.constraints.append(c)

    def pop(self):
        self.constraints.pop()

    def vars(self):
        return self.maximized.vars()

class ExpressionBuilder:
    """
    Represents a intermediate step in building a constraint/objective function
    """
    def __init__(self, lookup: dict[str, int]) -> None:
        self.lookup = lookup

    def __add__(self, other):
        if isinstance(other, PositiveVariable):
            copy = self.lookup.copy()
            copy[other.name] = copy.get(other.name, 0) + other.coef

            return ExpressionBuilder(copy)

    def __str__(self):
        out = []
        for key, value in self.lookup.items():
            out.append(f'{value} {key}')

        return ' + '.join(out)
    
    def __le__(self, other: float):
        return Constraint(self.lookup, other)
    
    def __ge__(self, other: float):
        return Constraint(self.lookup, other, gt=True)

    def to_maximized(self):
        return Maximized(self.lookup, 0.0)

class PositiveVariable:
    """
    Represents a real/integer variable that is positive
    """
    def __init__(self, name: str, coef: int = 1):
        self.name = name
        self.coef = coef

    def __neg__(self):
        return PositiveVariable(self.name, -self.coef)

    def __rmul__(self, other: float):
        return PositiveVariable(self.name, other)
    
    def __add__(self, other):
        if isinstance(other, PositiveVariable):
            if self.name == other.name:
                return ExpressionBuilder({
                    self.name: self.coef + other.coef
                })

            return ExpressionBuilder({
                self.name: self.coef,
                other.name: other.coef
            })
        
        
        raise TypeError("nope")
    
    def __sub__(self, other):
        if isinstance(other, PositiveVariable):
            return ExpressionBuilder({
                self.name: self.coef,
                other.name: -other.coef
            })
        
        raise TypeError("nope")
    
    def __str__(self):
        return f'{self.coef} {self.name}'

    def to_expression_builder(self):
        return ExpressionBuilder({
            self.name: self.coef
        })
    
    def __le__(self, other: float):
        return self.to_expression_builder() <= other

    def __ge__(self, other: float):
        return self.to_expression_builder() >= other

    def to_maximized(self):
        return Maximized({self.name: self.coef}, 0.0)


def makevars(text: str) -> list[PositiveVariable]:
    """
    Creates a list of positive variables with a space seperated list of names
    """
    vars = text.split(' ')

    out = []
    for var in vars:
        out.append(PositiveVariable(var))

    return out

# x = PositiveVariable('x')
# y = PositiveVariable('y')

# print(2 * x + 3* y + 0.1 * x <= 2)

In [ ]:
class Expression:
    CONSTANT = '@'

    lookup: dict[str, float]
    right: dict[str, float] = {}

    def __init__(self, left: dict[str, float], right: dict[str, float] = None) -> None:
        if right is None:
            right = {}
        
        self.lookup = left
        self.right = right

    def arg(self):
        for key in self.right:
            return key

    def substitute(self, other: 'Expression'):

        key = other.arg()

        while True:
            for k, v in self.lookup.items():
                if k == key:
                    # substitute
                    for ok, ov in other.lookup.items():
                        self.lookup[ok] = self.lookup.get(ok, 0) + v * ov

                    # remove the substituted
                    del self.lookup[k]
                    break

            else:  # if no breaks, ie, no replaces
                break  # we are done

    def rearrange_for(self, key: str):
        """
        Shift leftwards and move the key to the right
        """

        self.shift()

        if key not in self.lookup:
            raise ValueError('cannot find the key in the expression')
        
        # find the value
        value = self.lookup[key]
        del self.lookup[key]

        # divide everything on the left
        for k in self.lookup.keys():
            self.lookup[k] /= -value

        # move the value to the right, normalized
        self.right[key] = 1

    def shift(self):
        """
        Shift leftwards
        """

        for var, coef in self.right.items():
            if var not in self.lookup:
                self.lookup[var] = -coef
            else:
                self.lookup[var] -= coef

        self.right = {}


    def coefficients(self):
        coefs = []
        for key, value in self.lookup.items():
            if key != self.CONSTANT:
                coefs.append(value)

        return coefs
    

    def keys(self):
        data = []
        for key, value in self.lookup.items():
            if key != self.CONSTANT:
                data.append(key)

        return data
    

    def vars(self):
        data = []
        for key, value in self.lookup.items():
            if key != self.CONSTANT:
                data.append((key, value))

        return data
    

    def __str__(self) -> str:
        out = []
        for key, value in self.lookup.items():
            out.append(f"{value} {key}")
        
        left = ' + '.join(out)

        if len(self.right) == 0:
            return left
        
        out = []
        for key, value in self.right.items():
            out.append(f"{value} {key}")
        
        right = ' + '.join(out)

        return f"{left} = {right}"



# TODO! When i have time, complete a better, (>=, =) supporting solver using the tableau row reduction method
# With that implemented, the integer solver will work instantly
def betterSolveLP(problem: LPProblem):
    pass


def solveLP(problem: LPProblem, verbose=False) -> tuple[float, dict[str, float]]:
    """
    Solves the linear programming problem using the simplex method, assuming that the variables are real valued
    """

    # convert to expressions
    expressions: list[Expression] = []
    extras = []
    for i, const in enumerate(problem.constraints):
        loose = f's{i}'

        left = const.left.copy()
        left[loose] = 1  # because lhs <= rhs => lhs + s1 = rhs, for s1 >= 0, lhs >= 0
        left[Expression.CONSTANT] = -const.right  # shift constant leftwards

        # implicit (= 0)
        exp = Expression(
            left
        )
        exp.rearrange_for(loose)

        expressions.append(exp)


    # convert maximizer to expression
    output = problem.maximized.expression.copy()
    output[Expression.CONSTANT] = problem.maximized.constant
    maximizer = Expression(output)

    if verbose:
        for exp in expressions:
            print(exp)

        print(maximizer)

    # start the simplex method
    # assume (0,0) holds
    while True:

        # exits when no improvements can be made
        coefs = maximizer.coefficients()
        if all(map(lambda c: c <= 0, coefs)):
            break
        
        # pick the largest coef, but try to elimate all M
        vars = maximizer.vars()
        (key, _) = max(vars, key=lambda v: v[1])

        if verbose:
            print(f'loosening {key}')

        # loosen key, find the smallest non-positive ratio in the constraints
        loosened = key

        highest = -1e9
        index = -1
        for i, const in enumerate(expressions):
            if loosened not in const.keys():
                continue
            
            # we want to maximize the constant to coef ratio
            ratio = const.lookup.get(Expression.CONSTANT, 0) / const.lookup[loosened]
            if ratio > 0:  # this mf, zero is not positive dumbass
                continue
            
            if ratio > highest:
                highest = ratio
                index = i

        
        if index == -1:
            raise RuntimeError('cannot find a suitable tight alternative')
        
        # the anchor expression
        exp = expressions[index]

        if verbose:
            print('with expression:')
            print(exp)

        
        # rearange for the loose variable
        exp.rearrange_for(loosened)

        if verbose:
            print('rearranged:')
            print(exp)

        # and use exp to substitute the previously strict variables on other expressions
        for i, const in enumerate(expressions):
            if i == index:
                continue

            const.substitute(exp)

        # substitue the previously strict variable in the optimizer
        maximizer.substitute(exp)

        if verbose:
            print('updated')
            for exp in expressions:
                print(exp)

            print(maximizer)

    # the best score and vertex coordinate is found
    # by setting all strict variables to zero, ie, the constants

    maximized = maximizer.lookup[Expression.CONSTANT]
    parameters = {}
    for const in expressions:
        # we are skipping the introduced, s, loose variables
        key = const.arg()
        if key.startswith('s'):
            continue

        parameters[key] = const.lookup[Expression.CONSTANT]
    
    
    return maximized, parameters


# use the sample problem

# x0 = PositiveVariable('x0')
# x1 = PositiveVariable('x1')

# constraints = [
#     x0 <= 3000,
#     x1 <= 4000,
#     x0 + x1 <= 5000,
#     -x0 + 2 * x1 <= 2000
# ]
# m = (1.2 * x0 + 1.7 * x1).to_maximized()

# problem = LPProblem(constraints, m)
# print(problem)
# print('')

# best, param = solveLP(problem)
# print('Solution:')
# print(f'score = {best}')
# print(param)


      
x0 = PositiveVariable('x0')
x1 = PositiveVariable('x1')
x2 = PositiveVariable('x2')

# x0=0, x1=1.1
constraints = [
    x0 <= 1,
    x1 >= 1.1,
    x0 + x1 <= 2
]
m = (2 * x0 + 1 * x1).to_maximized()

problem = LPProblem(constraints, m)
print(problem)
print('')

best, param = solveLP(problem, verbose=True)
print('Solution:')
print(f'score = {best}')
print(param)

In [ ]:
# the above version doesn't work for some certain cases, in that it ignores some constraints

# here we devise a better version using theory


In [ ]:
x0 = PositiveVariable('x0')
x1 = PositiveVariable('x1')
x2 = PositiveVariable('x2')
x3 = PositiveVariable('x3')

constraints = [
    x0 <= 1,
    x1 <= 1,
    x2 <= 1,
    x3 <= 1,
    25 * x0 + 15 * x1 + 30 * x2 + 29 * x3 <= 40
]
m = (2 * x0 + 7 * x1 + 3 * x2 + 4 * x3).to_maximized()

problem = LPProblem(constraints, m)
print(problem)
print('')

best, param = solveLP(problem)
print('Solution:')
print(f'score = {best}')
print(param)

In [ ]:
x,y,z = makevars('x y z')

c = [
    3 * x + 2 * y + z <= 10,
    2 * x + 5 * y + 3 * z <= 15
]
m = (2 * x + 3 * y + 4 * z).to_maximized()

best, param = solveLP(LPProblem(c, m))
print(best)
print(param)

In [ ]:
import math

def solveIntLP(problem: LPProblem):
    """
    Solves the linear program using repeated applications of the simplex method,
    assuming that the variables must be integer-valued
    """

    # guesses initial values to be (0.5, 0.5, 0.5, ...)
    vars = problem.vars()
    guesses = [0] * len(vars)

    cycle = 0
    # just try every integer lmao
    while True:
        # choose a variable to optimize
        var = vars[cycle]
        value = guesses[cycle]
        lowerbound = value
        upperbound = value + 1

        # add constraints
        # v <= lower
        lowerconst = Constraint({var: 1}, lowerbound)
        # v >= upper
        upperconst = Constraint({var: 1}, upperbound, gt=True)

        problem.add(lowerconst)
        best1, p1 = solveLP(problem)
        problem.pop()

        
        problem.add(upperconst)
        best2, p2 = solveLP(problem)
        problem.pop()

        if best1 == best2:
            cycle = (cycle + 1) % len(vars)
            continue

        best = None
        sol = None
        if best1 > best2:
            problem.add(lowerconst)
            guesses[cycle] = max(0, guesses[cycle]-1)

            sol = p1
            best = best1
        else:
            problem.add(upperconst)
            guesses[cycle] += 1

            sol = p2
            best = best2

        # return
        cycle = (cycle + 1) % len(vars)

        # found an integer
        if all(map(lambda v: v - math.floor(v) < 0.001, sol.values())) and not all(map(lambda v: v == 0, sol.values())):
            return best, sol
        
x0 = PositiveVariable('x0')
x1 = PositiveVariable('x1')
x2 = PositiveVariable('x2')
x3 = PositiveVariable('x3')

constraints = [
    x0 <= 1,
    x1 <= 1,
    x2 <= 1,
    x3 <= 1,
    25 * x0 + 15 * x1 + 30 * x2 + 29 * x3 <= 40
]
m = (2 * x0 + 7 * x1 + 3 * x2 + 4 * x3).to_maximized()

problem = LPProblem(constraints, m)
print(problem)
print('')

best, param = solveLP(problem)
print('Solution:')
print(f'score = {best}')
print(param)
